In [39]:
import ee
import geemap.core as geemap
from geemap import geojson_to_ee

In [40]:
# authenticate
ee.Initialize()
ee.Authenticate()


Successfully saved authorization token.


In [18]:
# Demonstrate extracting pixels from an image as features with
# ee.Image.sample(), and show how the features are aligned with the pixels.
# read geojson
region = geojson_to_ee('./LT.geojson')

# An image with one band of elevation data.
image = ee.ImageCollection('COPERNICUS/S2')
image = image.filterDate('2019-06-01', '2019-06-30')
image = image.filterBounds(region).first()
vis_min = 1620
vis_max = 1650
m = geemap.Map()
m.add_layer(image, {'min': vis_min, 'max': vis_max}, 'COPERNICUS')


# Show region on the map.
m.centerObject(region, 10)

m.add_layer(region.style(color='00FF0022'))

# Generate 50 random points with the region.
random_points = ee.FeatureCollection.randomPoints(
    region=region, points=50, seed=0, maxError=1
)

display('Random points from within the defined region', random_points)
m = geemap.Map()
m.add_layer(random_points, {'color': 'black'}, 'Random points')
m

'Random points from within the defined region'

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [20]:
from geemap import ee_to_geojson

# Save these to a geoojson file
ee_to_geojson(random_points, filename='./random_points.geojson')

In [42]:
import json


points = {}
file = './random_points.geojson'
with open(file) as f:
    points = json.load(f)

m = geemap.Map()

# for each point
for pointDict in points['features']:
    # point to rectangle
    point = geojson_to_ee(pointDict)
    roi = point.buffer(1000).bounds()
    m.add_layer(roi)

    # get data
    # filter by roi, date, cloudiness
    # collection = ee.ImageCollection('COPERNICUS/S2') \
    #     .filterBounds(roi) \
    #     .filterDate('2021-01-01', '2024-05-01') \
    #     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    # m.addLayer(collection.first().clip(roi), {'min': 0, 'max': 3000}, 'COPERNICUS')
    m.centerObject(roi)

region = geojson_to_ee('./LT.geojson')
m.centerObject(region, 10)

m


Map(center=[55.324616240123405, 23.898186329460238], controls=(ZoomControl(options=['position', 'zoom_in_text'…